## This notebook generates the upper limit plot
#### Upper limits are precomputed and stored as dictionary. To compute these upper limits please refer to the other notebooks. 

In [2]:
import pycbc
import numpy as np
import matplotlib.pyplot as plt
import scipy
import h5py
from pycbc import conversions
from pycbc import cosmology as pycbccosmology
from scipy.stats import loguniform, gaussian_kde
from scipy.interpolate import interp1d, interp2d, RegularGridInterpolator, LinearNDInterpolator
from scipy.integrate import trapz, cumtrapz, quad
from random import choices
from collections import Counter
import seaborn as sns
import matplotlib as mpl
from matplotlib import rc
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.patches as patches
from decimal import Decimal
from astropy import cosmology
import subprocess
import astropy
from pycbc.population.population_models import *
from tqdm import tqdm
from matplotlib.lines import Line2D

sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')

mpl.rcParams['figure.figsize'] = [20.0, 7.0]
plt.rcParams['figure.dpi'] = 600
mpl.rcParams['font.size'] = 18

rc('font', family='serif', weight = 'bold')
rc('text', usetex=True)

colors = sns.color_palette("husl", 8)

### Table of predicted local merger rates and the estimated fraction of sources with e > 0.01 

In [3]:
predicted_rate_list = {'Belczynski et al. (2018a) (isolated binary)': np.array([8.0,10.,50.0]),
                'Arca Sedda (2020b) (globular cluster)': np.array([0.1]),
                'Trani et al. (2021) (triples)': np.array([0.04, 0.34, 0.11]),
                'Fragione et al. (2019) (nuclear cluster)': np.array([0.06, 0.1])
                      }

fraction_sources = {'Belczynski et al. (2018a) (isolated binary)': 0.00135,
                'Arca Sedda (2020b) (globular cluster)': 0.057397,
                'Trani et al. (2021) (triples)': 0.677439,
                'Fragione et al. (2019) (nuclear cluster)': 0.878149
                      }

### Observational upper limits and the same for limits for systems with $e_{10} \geq 0.01$

In [6]:
full_upper_limit_based_on_obs = {'Belczynski et al. (2018a) (isolated binary)':  149.02243058810643,
                        'Arca Sedda (2020b) (globular cluster)': 53.36526395563427,
                        'Trani et al. (2021) (triples)': 97.976810617951,
                        'Fragione et al. (2019) (nuclear cluster)': 70.44463776334753
                              }

ecc_upper_limit_based_on_obs = {'Belczynski et al. (2018a) (isolated binary)': 152200.0061,
                'Arca Sedda (2020b) (globular cluster)': 283.1574,
                'Trani et al. (2021) (triples)': 3.2827,
                'Fragione et al. (2019) (nuclear cluster)': 4.0228
                        }

### Upper limits for an idealized search with networks of current and future detectors 
#### For each astrophysical population we plot three different limits for:
* full population 
* systems with fixed $e_{10} \geq 0.01$
* systems with measurable eccentricities

In [4]:
full_upper_limit_O3_syn = {'Belczynski et al. (2018a) (isolated binary)': 139.64471793029105,
                           'Arca Sedda (2020b) (globular cluster)': 29.91369522966498,
                           'Trani et al. (2021) (triples)': 20.795764661815067,
                           'Fragione et al. (2019) (nuclear cluster)': 29.048940464177598}


fixed_upper_limit_O3_syn = {'Belczynski et al. (2018a) (isolated binary)': 222075.0, 
                            'Arca Sedda (2020b) (globular cluster)': 891.8674698795181,
                            'Trani et al. (2021) (triples)': 30.910600306217813,
                            'Fragione et al. (2019) (nuclear cluster)': 33.320294079821664}



measurable_upper_limit_O3_syn = {'Belczynski et al. (2018a) (isolated binary)': 6120.0,
                                'Arca Sedda (2020b) (globular cluster)': 4613.0,
                                 'Trani et al. (2021) (triples)': 995.9,
                                 'Fragione et al. (2019) (nuclear cluster)': 95.98}



full_upper_limit_CE_only = {'Belczynski et al. (2018a) (isolated binary)': 0.0021919213943572277, 
                             'Arca Sedda (2020b) (globular cluster)': 0.0011257530053556922, 
                             'Trani et al. (2021) (triples)': 0.0010080234416283082,
                             'Fragione et al. (2019) (nuclear cluster)': 0.001082967826882465}


fixed_upper_limit_CE_only = {'Belczynski et al. (2018a) (isolated binary)': 2.6722573839662442,
                             'Arca Sedda (2020b) (globular cluster)': 0.034350762054564186,
                             'Trani et al. (2021) (triples)': 0.0014993702562062906,
                             'Fragione et al. (2019) (nuclear cluster)': 0.0012444783733012645}


measurable_upper_limit_CE_only = {'Belczynski et al. (2018a) (isolated binary)': 1.712,
                                  'Arca Sedda (2020b) (globular cluster)': 0.06979,
                                  'Trani et al. (2021) (triples)': 0.009821, 
                                  'Fragione et al. (2019) (nuclear cluster)': 0.004279}


full_upper_limit_asharp = {'Belczynski et al. (2018a) (isolated binary)': 0.48555766392578537,
                           'Arca Sedda (2020b) (globular cluster)': 0.10645969508981815,
                           'Trani et al. (2021) (triples)': 0.0778671896131999,
                           'Fragione et al. (2019) (nuclear cluster)': 0.10731093611667676}


fixed_upper_limit_asharp = {'Belczynski et al. (2018a) (isolated binary)': 677.3529411764706,
                            'Arca Sedda (2020b) (globular cluster)': 3.330922765403529,
                            'Trani et al. (2021) (triples)': 0.1157658744520851,
                            'Fragione et al. (2019) (nuclear cluster)': 0.12335693701993637}

measurable_upper_limit_asharp = {'Belczynski et al. (2018a) (isolated binary)': 1279.0,
                                 'Arca Sedda (2020b) (globular cluster)': 8.53,
                                 'Trani et al. (2021) (triples)': 0.7375,
                                 'Fragione et al. (2019) (nuclear cluster)': 0.2632}

### Plot the estimated upper limits for the different detector configurations along with the modelled predicted merger rates.

In [ ]:
fig, axs = plt.subplots()
colors = ['springgreen', 'hotpink']

xmin = 8e-4
xmax = 2e3
ymax = 28

count = 1
diff_mult = 5
key_index = 0
k = 0
time_req = []
limit_array = []
model_colors = iter(cm.RdPu(np.linspace(0.6,1,4)))

for key in predicted_rate_list.keys():
    #print(count, key)
    #Current detectors fractional_rate
    fractional_rate = np.array(predicted_rate_list[key])#*fraction_sources[key]
    
    #Time required by future-gen detect 
    extra_vdist = 1
    size = 30
    if key == 'Belczynski et al. (2018a) (isolated binary)':
        color = 'springgreen'
    else:
        color = next(model_colors)

    if len(fractional_rate) > 1:
        rate = np.logspace(np.log10(min(fractional_rate)), np.log10(max(fractional_rate)), 500)
        #Predicted rates
        axs.scatter(rate, np.full(len(rate), diff_mult*count - extra_vdist), s=size, color='black', zorder = 15)      
        
        #Side-blobs to the rates
        axs.scatter(np.array([min(rate), max(rate)]), np.full(2, diff_mult*count - extra_vdist), s=size*6, color='black', zorder = 15)
        axs.scatter(np.array([min(rate), max(rate)]), np.full(2, diff_mult*count - extra_vdist), s=size, color='white', zorder = 15)
        axs.scatter(np.full(20,min(rate)), np.linspace(5*count - extra_vdist*0.4, diff_mult*count - extra_vdist*1.6, 20), s=4, color='white',zorder = 15)
        axs.scatter(np.full(20,max(rate)), np.linspace(5*count - extra_vdist*0.4, diff_mult*count - extra_vdist*1.6, 20), s=4, color='white',zorder = 15)

        #Obesrvational upper limits
        upper_limit = ecc_upper_limit_based_on_obs[key]
        limit_array.append(upper_limit)
        axs.text(max(fractional_rate)*1.4, diff_mult*count - extra_vdist, key, verticalalignment='center', fontsize=18, zorder=6, bbox=dict(facecolor='white', edgecolor='none', pad=1))
    else:
#        axs.text(fractional_rate*1.4, diff_mult*count-0.5, key, verticalalignment='center', fontsize='medium')
        axs.text(fractional_rate*1.4, diff_mult*count - extra_vdist, key, verticalalignment='center', fontsize=18, bbox=dict(facecolor='white', edgecolor='none', pad=5))

        axs.scatter(fractional_rate, diff_mult*count - extra_vdist, marker='|', s=200, color='black', zorder = 15)
        axs.scatter(fractional_rate*1.08, diff_mult*count - extra_vdist, marker='<', s=200, color='black', zorder = 15)

        
    if key_index == 0:
        count += 1
        plt.axhline(5*count, linestyle='dotted',color='gray')
        axs.text(1.2*xmin, diff_mult*count-3, 'BNS', fontsize='xx-large', horizontalalignment='left', verticalalignment='center', color='gray')
        k += 1
        axs.text(1.2*xmin, diff_mult*count+2, 'NSBH', fontsize='xx-large', horizontalalignment='left', verticalalignment='center', color='gray')
        #count += 1x = np.linspace(0, 10, 400)

    count += 1 
    key_index += 1

    
# Req time lines
time_req_det = {}
limits_det = {}
hatch_list = ['x', 'o', '.', '--']
#det_list = ['Current observations', 'A+', 'A\#', 'CE-CE-ET', 'CE', 'syn_O3', 'O3-measurable', '3G-measurable', 'asharp-measurable']
#det_list = ['O3-observations','O3-full', 'O3-fixed', 'CE-full', 'CE-fixed', 'CE-measurable']
det_list = ['Current observations','O3 theoretical', 'asharp full', 'asharp fixed', 'asharp measurable', 'CE full',  'CE fixed', 'CE measurable']


k=0
#det_colors = iter(cm.GnBu(np.linspace(0.3, 1, len(det_list))))
det_colors = iter(['cornflowerblue','green','deeppink'])
det_colors_measurable = iter(['green', 'deeppink'])
det_colors_fixed = iter(['green', 'deeppink'])


for det in det_list:
    time_req = np.array([])
    limit_array = []
    
    if det == 'Current observations':
        ul_array = full_upper_limit_based_on_obs
    elif det == 'O3 theoretical':
        ul_array = full_upper_limit_O3_syn
    elif det == 'O3 e >= 0.01':
        ul_array = fixed_upper_limit_O3_syn
    elif det == 'O3-measurable':
        ul_array = measurable_upper_limit_O3_syn

    elif det == 'CE full':
        ul_array = full_upper_limit_CE_only
    elif det == 'CE fixed':
        ul_array = fixed_upper_limit_CE_only
    elif det == 'CE measurable':
        ul_array = measurable_upper_limit_CE_only
        
    elif det == 'asharp full':
        ul_array = full_upper_limit_asharp
    elif det == 'asharp fixed':
        ul_array = fixed_upper_limit_asharp
    elif det == 'asharp measurable':
        ul_array = measurable_upper_limit_asharp


    count = 1
    key_index = 0
    y_values = []
    #print(det, k)
    for key in predicted_rate_list.keys():
        upper_limit = ul_array[key]
        #print(key, upper_limit)
        
        if key_index == 0:
            y_values.append(0)
            limit_array.append(upper_limit)
        
        #Transition form BNS to NSBH
        if key_index == 0:
            y_values.append(diff_mult*(count-4) - extra_vdist + diff_mult/2 )
            limit_array.append(upper_limit)
            
            ##Labelling the detector lines
            #if det == 'observations':
            #    axs.text(limit_array[-1]*1.1, 1.2*count, det, fontsize='small', rotation=90)
            #elif det != 'Current observations':
            #    axs.text(limit_array[-1]*0.9, 1.2*count, det, fontsize='small', rotation=90)
        
        #lower edge of the box
        y_values.append(diff_mult*count - extra_vdist - diff_mult/2)
        limit_array.append(upper_limit)
        
        #Og limit 
        y_values.append(diff_mult*count - extra_vdist)
        limit_array.append(upper_limit)
        axs.scatter(upper_limit, diff_mult*count - extra_vdist, marker='.', color='black')
        
        #Upper edge of the box
        y_values.append(diff_mult*count - extra_vdist + diff_mult/2)
        limit_array.append(upper_limit)
        
        if key_index == 3:
            y_values.append(ymax)
            limit_array.append(upper_limit)
        
        if key_index == 0:
            count += 1

        #Time required 
        obs_time = 1
        #time_req = np.concatenate([time_req, obs_time*upper_limit/fractional_rate*dist_factor**(-3) ])
        count += 1
        key_index += 1
        
    if det == 'O3 theoretical' or det == 'CE full' or det == 'asharp full': 
        c = next(det_colors)
        axs.plot(limit_array, y_values, c=c)
    else:
        print('Not plotting for', det)
        axs.plot(limit_array, y_values,'--', linewidth=0.5)

        if det == 'asharp measurable' or det == 'CE measurable':
            bm = next(det_colors_measurable)
            axs.plot(limit_array, y_values, c=bm, linewidth=1.5, linestyle=':')
        elif det == 'asharp fixed' or det == 'CE fixed':  
            bf = next(det_colors_fixed)
            axs.plot(limit_array, y_values, c=bf, linewidth=1.5, linestyle='--')

    #time_req_det[det] = ((min(time_req), max(time_req)))
    limits_det[det] = (limit_array, y_values)
    k +=1
    
    
#Shade different det_configs
#axs.fill_betweenx(y_values, limits_det['O3 theoretical'][0], xmax, alpha=0.2, hatch = '.', facecolor='white', label='Current detectors theoretical')
axs.fill_betweenx(y_values, limits_det['Current observations'][0], xmax, hatch='/', facecolor='cornflowerblue', label='Current observations')
axs.fill_betweenx(y_values, limits_det['CE fixed'][0], limits_det['CE measurable'][0], hatch='', facecolor='lightpink', label='CE(40km) + CE(20km)')
axs.fill_betweenx(y_values, limits_det['asharp fixed'][0], limits_det['asharp measurable'][0], hatch='', facecolor='lightgreen', label='three A\# detectors')

#Put arrowsright. Current observational limits could be reduced by
axs.annotate('',xy=(1.1*limits_det['O3 theoretical'][0][9], y_values[9]),
             xytext=(limits_det['Current observations'][0][9], y_values[9]),
             arrowprops=dict(arrowstyle='->'))
axs.annotate('',xy=(1.1*limits_det['O3 theoretical'][0][6], y_values[6]),
             xytext=(limits_det['Current observations'][0][6], y_values[6]),
             arrowprops=dict(arrowstyle='->'))
axs.annotate('',xy=(1.1*limits_det['O3 theoretical'][0][12], y_values[12]),
             xytext=(limits_det['Current observations'][0][12], y_values[12]),
             arrowprops=dict(arrowstyle='->'))


## Labelling some detector lines
#axs.text(limits_det['syn_O3'][0][0]*0.8, 1.2*2, 'Current limits \n using simulations', fontsize='medium', rotation=90)
#axs.text(limits_det['O3-measurable'][0][0]*0.8, 1.2*2, 'O3-measurable', fontsize='medium', rotation=90)
#axs.text(limits_det['3G-measurable'][0][0]*0.8, 1.2*2, '3G-measurable', fontsize='medium', rotation=90)


axs.yaxis.set_visible(False)
locmaj = ticker.LogLocator(base=10, numticks=15)
major_tick_locations = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000, 10000]
axs.xaxis.set_major_locator(ticker.FixedLocator(major_tick_locations))
axs.xaxis.set_major_formatter(ticker.ScalarFormatter())
axs.tick_params(which='major', width=2, length=8) 
axs.tick_params(which='minor', width=2, length=5) 


axs.set_xscale('log')
axs.set_xlabel('Upper limits on the local merger rate ($\mathrm{Gpc}^{-3} \mathrm{Yr}^{-1}$) for BNS and NSBH systems', fontsize=22)


axs.set_ylim(0,ymax)
axs.set_xlim(xmin,xmax)
axs.xaxis.grid(alpha=0.5, color='grey')

## Lgend stuff
custom_lines = [Line2D([0], [0], color='black', lw=2),
               Line2D([0], [0], color='black', linestyle = '--', lw=2),
               Line2D([0], [0], color='black', linestyle = ':', lw=2)]
custom_labels = ['Full population', '$e_{10} >= 0.01$', 'Measurable eccentricity']

lines, labels = plt.gca().get_legend_handles_labels()
lines += custom_lines
labels += custom_labels

plt.legend(lines, labels, loc=1)
plt.tight_layout()
#plt.savefig('final_plots/expected_ranges.png', dpi=600)
#plt.savefig('TaylorF2ecc-limits.png', dpi=600)
plt.show()

Not plotting for Current observations
Not plotting for asharp fixed
Not plotting for asharp measurable
Not plotting for CE fixed
Not plotting for CE measurable
